This is an introductory tutorial to the main features of plangym.

## Working with states

### `reset` and `step` return the environment state

The main difference with the `gymnasium` API is that environment state is considered as important as observations, rewards and terminal flags. This is why plangym incorporates them to the tuples that the environment returns after calling `step` and `reset`:

- The `reset` method will return a tuple of (state, observation, info) unless you pass `return_state=False` as an argument.

- When `step` is called passing the environment state as an argument it will return a tuple containing `(state, obs, reward, terminated, truncated, info)`

In [ ]:
import plangym

env = plangym.make("CartPole-v1")
action = env.action_space.sample()

state, obs, info = env.reset()
state, obs, reward, terminated, truncated, info = env.step(action, state)

However, if you don't provide the environment state when calling `step`, the returned tuple will match the standard `gymnasium` interface:

In [ ]:
env = plangym.make("CartPole-v1")
action = env.action_space.sample()

obs, info = env.reset(return_state=False)
obs, reward, terminated, truncated, info = env.step(action)

### Accessing and modifying the environment state

You can get a copy of the environment's state calling `env.get_state()`:

In [3]:
state = env.get_state()
state

array([ 0.03145539,  0.17749025,  0.01348916, -0.25611924])

And set the environment state using `env.set_state(state)`

In [4]:
env.set_state(state)
assert (state == env.get_state()).all()

## Step vectorization

All plangym environments offer a `step_batch` method that allows vectorized steps of batches of states and actions. 

Calling `step_batch` with a list of states and actions will return a tuple of lists containing the step data for each of the states and actions provided.

In [ ]:
states = [state.copy() for _ in range(10)]
actions = [env.action_space.sample() for _ in range(10)]

data = env.step_batch(states=states, actions=actions)
new_states, observs, rewards, terminateds, truncateds, infos = data
type(new_states), type(observs)

### Parallel step vectorization using multiprocessing

Passing the argument `n_workers` to `plangym.make` will return an environment that steps a batch of actions and states in parallel using multiprocessing.

In [ ]:
env = plangym.make("CartPole-v1", n_workers=2)
states = [state.copy() for _ in range(10)]
actions = [env.action_space.sample() for _ in range(10)]

data = env.step_batch(states=states, actions=actions)
new_states, observs, rewards, terminateds, truncateds, infos = data
type(env), type(new_states), type(observs)

### Step vectorization using ray

It is possible to use ray actors to step the environment in parallel when calling `step_batch` by passing `ray=True` to `plangym.make`

In [ ]:
import ray
ray.init()

env = plangym.make("CartPole-v1", n_workers=2, ray=True)
states = [state.copy() for _ in range(10)]
actions = [env.action_space.sample() for _ in range(10)]

data = env.step_batch(states=states, actions=actions)
new_states, observs, rewards, terminateds, truncateds, infos = data
type(env), type(new_states), type(observs)